# Negative Sampling  : 네거티브 샘플링

### Embedding과 EmbeddingDot 계층

In [1]:
import numpy as np

# nn_layers.py에 추가하여 놓는다

# Embedding 계층
class Embedding :
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    # 순전파
    def forward(self,idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    # 역전파
    def backward(self, dout):  # 중복 인덱스가 있어도 올바르게 처리, 속도가 빠름
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)  
        return None       

In [2]:
# EmbeddingDot 계층
# nn_layers.py에 추가하여 놓는다

class EmbeddingDot:
    def __init__(self,W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self,h,idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W*h,axis=1)   # 1차원 출력
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0],1) # 2차원으로 변환
        
        dtarget_W = dout*h  # sum <--> repeat, 브로드캐스트
        self.embed.backward(dtarget_W)
        
        dh = dout*target_W  # 브로드캐스트
        return dh

In [3]:
# EmbeddingDot 클래스 테스트 
W = np.arange(21).reshape(7,3)
print('W:\n',W)

idx = np.array([0,3,1])
print('idx:\n',idx)

h = W[[0,1,2]]
print('h:\n',h)

embed_dot = EmbeddingDot(W)
target_W = embed_dot.forward(h,idx)
print('target_W:\n',target_W)

W:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
idx:
 [0 3 1]
h:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
target_W:
 [  5 122  86]


In [4]:
# EmbeddingDot 계층의 forward 함수내에서 변수의 값 변화 정보
idx = np.array([0,3,1])
embed = Embedding(W)
target_W = embed.forward(idx) # W에서 임베딩 처리
print('target_W:\n',target_W)

h = W[[0,1,2]]
print('h:\n',h)

temp = target_W * h
print('temp:\n',temp)

out = np.sum(temp,axis=1)
print('out:\n',out)

target_W:
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]]
h:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
temp:
 [[ 0  1  4]
 [27 40 55]
 [18 28 40]]
out:
 [  5 122  86]
